0. Import stuff

1. Load model:  GCN + LSTM
    - a) The GCN class and functions it uses
    - b) Load the model
 
2. Load the data sets
    - that have been downloaded with the other notebook
    - a) training set
    - b) test set
        - The test set should have the same amount of KG context as in the training set
        
3. Training
    - a) Training function
    - b) Run the training
    - c) Plot the losses from training 

4. Testing
    - a) Test function
    - b) Call the function and display / save results

### 0

In [1]:
import os
import random
import requests
import numpy as np

from gensim.models import KeyedVectors
import logging

from gensim import utils
import nltk

import networkx as nx
import matplotlib.pyplot as plt

import shutil
import smart_open
import sys 

from sys import platform
import random
import tensorflow as tf
import json

import time

### 1

a)

In [19]:
TINY = 1e-6
ONE = tf.constant(1.)
NAMESPACE = 'gcn_qa'
forbidden_weight = 1.
_weight_for_positive_matches = 1.
_rw = 1e-1
tf.compat.v1.disable_eager_execution()
class GCN_QA(object):
    _nodes_vocab_size = 300
    _question_vocab_size = 300
    _question_vector_size = 150
    _types_size = 3
    _mask_size = 200
    _types_proj_size = 5
    _word_proj_size = 50
    _word_proj_size_for_rnn = 50
    _word_proj_size_for_item = 50
    _internal_proj_size = 250
    _hidden_layer1_size = 250
    _hidden_layer2_size = 250
    _output_size = 2

    _memory_dim = 100
    _stack_dimension = 2

    def __init__(self, dropout=1.0):
        tf.compat.v1.reset_default_graph()
        with tf.compat.v1.variable_scope(NAMESPACE):
            config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
            self.sess = tf.compat.v1.Session(config=config)

            # Input variables
            self.node_X = tf.compat.v1.placeholder(tf.float32, shape=(None, None, self._nodes_vocab_size), name='node_X')
            self.types = tf.compat.v1.placeholder(tf.float32, shape=(None, None, self._types_size), name='types')
            self.Wt = tf.Variable(tf.random.uniform([self._types_size,
                                                     self._types_proj_size], -_rw, _rw))
            self.bt = tf.Variable(tf.random.uniform([self._types_proj_size], -_rw, _rw))
            #problem tf.nn
            
            self.types_projection = lambda x: tf.nn.relu(tf.matmul(x, self.Wt) + self.bt)
            self.types_internal = tf.map_fn(self.types_projection, self.types)
            self.question_vectors_fw = tf.compat.v1.placeholder(tf.float32, shape=(None, None, self._question_vocab_size),
                                                      name='question_vectors_inp_fw')
            self.question_vectors_bw = tf.compat.v1.placeholder(tf.float32, shape=(None, None, self._question_vocab_size),
                                                      name='question_vectors_inp_nw')
            self.question_mask = tf.compat.v1.placeholder(tf.float32, shape=(None, None, self._mask_size),
                                                name='question_mask')

            # The question is pre-processed by a bi-GRU
            self.Wq = tf.Variable(tf.random.uniform([self._question_vocab_size,
                                                     self._word_proj_size_for_rnn], -_rw, _rw))
            self.bq = tf.Variable(tf.random.uniform([self._word_proj_size_for_rnn], -_rw, _rw))
            self.internal_projection = lambda x: tf.nn.relu(tf.matmul(x, self.Wq) + self.bq)
            self.question_int_fw = tf.map_fn(self.internal_projection, self.question_vectors_fw)
            self.question_int_bw = tf.map_fn(self.internal_projection, self.question_vectors_bw)
            
            #problem rnn
            self.rnn_cell_fw = tf.compat.v1.nn.rnn_cell.MultiRNNCell([tf.compat.v1.nn.rnn_cell.GRUCell(self._memory_dim) for _ in range(self._stack_dimension)],
                                                state_is_tuple=True)
            self.rnn_cell_bw = tf.compat.v1.nn.rnn_cell.MultiRNNCell([tf.compat.v1.nn.rnn_cell.GRUCell(self._memory_dim) for _ in range(self._stack_dimension)],
                                                state_is_tuple=True)
            with tf.compat.v1.variable_scope('fw'):
                #problem dynamic rnn
                output_fw, state_fw = tf.compat.v1.nn.dynamic_rnn(self.rnn_cell_fw, self.question_int_fw, time_major=True,
                                                        dtype=tf.float32)
            with tf.compat.v1.variable_scope('bw'):
                #problem dynamen rnn
                output_bw, state_bw = tf.compat.v1.nn.dynamic_rnn(self.rnn_cell_bw, self.question_int_bw, time_major=True,
                                                        dtype=tf.float32)

            self.states = tf.concat(values=[output_fw, tf.reverse(output_bw, [0])], axis=2)
            self.question_vector_pre = tf.reduce_mean(tf.multiply(self.question_mask, self.states), axis=0)
            self.Wqa = tf.Variable(
                tf.random.uniform([2 * self._memory_dim, self._question_vector_size], -_rw, _rw),
                name='Wqa')
            self.bqa = tf.Variable(tf.random.uniform([self._question_vector_size], -_rw, _rw), name='bqa')
            #problem tf.nn
            self.question_vector = tf.nn.relu(tf.matmul(self.question_vector_pre, self.Wqa) + self.bqa)

            # Dense layer before gcn
            self.Wi = tf.Variable(tf.random.uniform([self._nodes_vocab_size,
                                                     self._word_proj_size], -_rw, _rw))
            self.bi = tf.Variable(tf.random.uniform([self._word_proj_size], -_rw, _rw))
            self.internal_projection2 = lambda x: tf.nn.relu(tf.matmul(x, self.Wi) + self.bi)
            self.word_embeddings = tf.map_fn(self.internal_projection2, self.node_X)

            self.inputs = tf.concat(values=[self.word_embeddings, self.types_internal], axis=2)
            self.Wp = tf.Variable(tf.random.uniform([self._word_proj_size + self._types_proj_size,
                                                     self._internal_proj_size], -_rw, _rw))
            self.bp = tf.Variable(tf.random.uniform([self._internal_proj_size], -_rw, _rw))
            self.enc_int_projection = lambda x: tf.nn.relu(tf.matmul(x, self.Wp) + self.bp)
            self.enc_int = tf.map_fn(self.enc_int_projection, self.inputs)

            # GCN part
            self.Atilde_fw = tf.nn.dropout(tf.compat.v1.placeholder(tf.float32, shape=(None, None, None), name="Atilde_fw"), 0.25)

            self.X1_fw = GCN_layer_fw(self._internal_proj_size,
                                      self._hidden_layer1_size,
                                      self._question_vector_size,
                                      self.enc_int,
                                      self.Atilde_fw,
                                      self.question_vector,
                                      '_1')
            #problem tf.nn
            self.X1_fw_dropout = tf.nn.dropout(self.X1_fw, dropout)

            self.X2_fw = GCN_layer_fw(self._hidden_layer1_size,
                                      self._hidden_layer1_size,
                                      self._question_vector_size,
                                      self.X1_fw_dropout,
                                      self.Atilde_fw,
                                      self.question_vector,
                                      '_2')
            self.X2_fw_dropout = tf.nn.dropout(self.X2_fw, dropout)

            self.X3_fw = GCN_layer_fw(self._hidden_layer1_size,
                                      self._hidden_layer1_size,
                                      self._question_vector_size,
                                      self.X2_fw_dropout,
                                      self.Atilde_fw,
                                      self.question_vector,
                                      '_3')
            self.X3_fw_dropout = tf.nn.dropout(self.X3_fw, dropout)

            self.X4_fw = GCN_layer_fw(self._hidden_layer1_size,
                                      self._hidden_layer1_size,
                                      self._question_vector_size,
                                      self.X3_fw_dropout,
                                      self.Atilde_fw,
                                      self.question_vector,
                                      '_4')
            #tf.nn
            self.X4_fw_dropout = tf.nn.dropout(self.X4_fw, dropout)
            self.first_node = self.X4_fw_dropout[0]
            self.concatenated = tf.concat(values=[self.question_vector, self.first_node], axis=1)

            # Final feedforward layers
            self.Ws1 = tf.Variable(
                tf.random.uniform([self._question_vector_size
                                   + self._hidden_layer1_size,
                                   self._hidden_layer2_size], -_rw, _rw),
                name='Ws1')
            self.bs1 = tf.Variable(tf.random.uniform([self._hidden_layer2_size], -_rw, _rw), name='bs1')
            #problem tf.nn
            self.first_hidden = tf.nn.relu(tf.matmul(self.concatenated, self.Ws1) + self.bs1)
            self.first_hidden_dropout = tf.nn.dropout(self.first_hidden, dropout)

            self.Wf = tf.Variable(
                tf.random.uniform([self._hidden_layer2_size, self._output_size], -_rw,
                                  _rw),
                name='Wf')
            self.bf = tf.Variable(tf.random.uniform([self._output_size], -_rw, _rw), name='bf')
            #problem tf.nn
            self.outputs = tf.nn.softmax(tf.matmul(self.first_hidden_dropout, self.Wf) + self.bf)

            # Loss function and training
            self.y_ = tf.compat.v1.placeholder(tf.float32, shape=(None, self._output_size), name='y_')
            self.outputs2 = tf.squeeze(self.outputs)
            self.y2_ = tf.squeeze(self.y_)
            self.one = tf.ones_like(self.outputs)
            self.tiny = self.one * TINY
            self.cross_entropy = (tf.reduce_mean(
                -tf.reduce_sum(self.y_ * tf.compat.v1.log(self.outputs + self.tiny) * _weight_for_positive_matches
                               + (self.one - self.y_) * tf.compat.v1.log(
                    self.one - self.outputs + self.tiny))
            ))

        # Clipping the gradient
        optimizer = tf.compat.v1.train.AdamOptimizer(1e-4)
        gvs = optimizer.compute_gradients(self.cross_entropy)
        capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs if var.name.find(NAMESPACE) != -1]
        self.train_step = optimizer.apply_gradients(capped_gvs)
        self.sess.run(tf.compat.v1.global_variables_initializer())

        # Adding the summaries
        tf.summary.scalar('cross_entropy', self.cross_entropy)
        self.merged = tf.compat.v1.summary.merge_all(key=tf.compat.v1.GraphKeys.SUMMARIES)
        self.train_writer = tf.compat.v1.summary.FileWriter('./train', self.sess.graph)

    def _add_identity(self, A):
        num_nodes = A.shape[0]
        identity = np.identity(num_nodes)
        return identity + A

    def __train(self, A_fw, node_X, types, item_vector, question_vectors, question_mask, y):
        item_vector = np.array(item_vector)
        Atilde_fw = np.array([self._add_identity(item) for item in A_fw])
        node_X = np.array(node_X)
        
        node_X = np.transpose(node_X, (1, 0, 2))

        types = np.array(types)
        types = np.transpose(types, (1, 0, 2))

        question_vectors = np.array(question_vectors)
        question_vectors_fw = np.transpose(question_vectors, (1, 0, 2))
        question_vectors_bw = question_vectors_fw[::-1, :, :]

        question_mask = np.array(question_mask)
        question_mask = np.transpose(question_mask, (1, 0, 2))

        y = np.array(y)

        feed_dict = {}
        feed_dict.update({self.node_X: node_X})
        feed_dict.update({self.types: types})
        feed_dict.update({self.question_vectors_fw: question_vectors_fw})
        feed_dict.update({self.question_vectors_bw: question_vectors_bw})
        feed_dict.update({self.question_mask: question_mask})
        feed_dict.update({self.Atilde_fw: Atilde_fw})
        feed_dict.update({self.y_: y})
        loss, _,outputs2, y2 = self.sess.run(
            [self.cross_entropy, self.train_step,  self.outputs2, self.y2_], feed_dict)
        return loss

    def train(self, data, epochs=20):
        losses = []
        
#         print(type(data[0][6]))
        for epoch in range(epochs):
        
            loss = self.__train([data[i][0] for i in range(len(data))],
                                   [data[i][1] for i in range(len(data))],
                                   [data[i][2] for i in range(len(data))],
                                   [data[i][3] for i in range(len(data))],
                                   [data[i][4] for i in range(len(data))],
                                   [data[i][5] for i in range(len(data))],
                                   [data[i][6] for i in range(len(data))])

#             print(loss)
            losses.append(loss)
            
            sys.stdout.flush()
        return losses

    def __predict(self, A_fw, node_X, types, item_vector, question_vectors, question_mask):
        item_vector = np.array(item_vector)
        Atilde_fw = np.array([self._add_identity(item) for item in A_fw])

        node_X = np.array(node_X)
        node_X = np.transpose(node_X, (1, 0, 2))

        types = np.array(types)
        types = np.transpose(types, (1, 0, 2))

        question_vectors = np.array(question_vectors)
        question_vectors_fw = np.transpose(question_vectors, (1, 0, 2))
        question_vectors_bw = question_vectors_fw[::-1, :, :]

        question_mask = np.array(question_mask)
        question_mask = np.transpose(question_mask, (1, 0, 2))

        feed_dict = {}
        feed_dict.update({self.node_X: node_X})
        feed_dict.update({self.types: types})
        feed_dict.update({self.question_vectors_fw: question_vectors_fw})
        feed_dict.update({self.question_vectors_bw: question_vectors_bw})
        feed_dict.update({self.question_mask: question_mask})
        feed_dict.update({self.Atilde_fw: Atilde_fw})

        y_batch = self.sess.run([self.outputs2], feed_dict)
        return y_batch

    def __standardize_item(self, item):
        if item[0] < item[1]:
            return [0., 1.]
        return [1., 0.]

    def predict(self, A_fw, node_X, types, item_vector, question_vectors, question_mask):
        output = self.__predict([A_fw], [node_X], [types], [item_vector], [question_vectors], [question_mask])
        return self.__standardize_item(output[0])

    # Loading and saving functions

    def save(self, filename):
        saver = tf.compat.v1.train.Saver()
        saver.save(self.sess, filename)

    def load_tensorflow(self, filename):
        saver =tf.compat.v1.train.Saver([v for v in tf.compat.v1.global_variables() if NAMESPACE in v.name])
        saver.restore(self.sess, filename)

    @classmethod
    def load(self, filename, dropout=0.25):
        model = GCN_QA(dropout)
        model.load_tensorflow(filename)
        return model

In [16]:
def compute_new_adjacency_matrix(embedding_size, attention_size, memory_dim, A, H, question_vector, name):
    Wa = tf.Variable(tf.compat.v1.random_uniform([embedding_size + memory_dim, attention_size], -_rw, _rw),
                     name='Wa_' + name)
    ba = tf.Variable(tf.compat.v1.random_uniform([attention_size], -_rw, _rw), name='b0_fw' + name)
    #problem with tf.nn
    WHQ_projection = lambda x: tf.nn.relu(tf.matmul(tf.concat([x, question_vector], axis=1), Wa) + ba)
    WHQ = tf.map_fn(WHQ_projection, H)
    WHQ = tf.transpose(WHQ, perm=[1, 0, 2])
    #problem with tf.nn
    WHQ_squared_projection = lambda x: tf.nn.softmax(tf.matmul(x, tf.transpose(x, perm=[1, 0])))
    WHQ_squared = tf.map_fn(WHQ_squared_projection, WHQ)
    new_A = tf.multiply(A, WHQ_squared)
    return new_A


def GCN_layer_fw(embedding_size, hidden_layer1_size, memory_dim, hidden, Atilde_fw, question_vector, name):
    new_A = compute_new_adjacency_matrix(embedding_size, 250, memory_dim, Atilde_fw, hidden, question_vector, name)
    W0_fw = tf.Variable(tf.compat.v1.random_uniform([embedding_size, hidden_layer1_size], -_rw, _rw),
                        name='W0_fw' + name)
    b0_fw = tf.Variable(tf.compat.v1.random_uniform([hidden_layer1_size], -_rw, _rw), name='b0_fw' + name)
    left_X1_projection_fw = lambda x: tf.matmul(x, W0_fw) + b0_fw
    left_X1_fw = tf.map_fn(left_X1_projection_fw, hidden)
    left_X1_fw = tf.transpose(left_X1_fw, perm=[1, 0, 2], name='left_X1_fw' + name)
    #problem with tf.nn
    X1_fw = tf.nn.relu(tf.matmul(new_A, left_X1_fw))
    X1_fw = tf.transpose(X1_fw, perm=[1, 0, 2])
    return X1_fw

b)

In [20]:
import random
import sys

nn_model_gcn_with_attention = GCN_QA(dropout=0.25)

### 2

a)

In [6]:
import pickle
# Store data (serialize)

# pickle file, saved from running the make_training_set_wikidata notebook
   # name has model, number of entities and number of nodes
current_training_set =  'training_data_gcn_188_10.pickle'

with open(current_training_set, 'rb') as handle:
    training_data = pickle.load(handle)

b)

In [23]:
training_data[0]

{'text': ' fantasy novelist David Gemmell. Achilles is featured heavily in the novel The Firebrand by Marion Zimmer Bradley. The comic book hero Captain Marvel is endowed with the courage of Achilles, as well ',
 'item': 'captain marvel',
 'wikidata_id': 'Q534153',
 'graph': {'A_bw': array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
          0.],
     

In [ ]:
# pickle file, saved from running the make_training_set_wikidata notebook
   # name has model, number of entities and number of nodes
    
current_test_set =  'test_data_gcn_2000_10.pickle'
with open(current_test_set, 'rb') as handle:
    test_set = pickle.load(handle)

### 3


In [7]:
def get_chunks(l, n):
    return [l[i:i + n] for i in range(0, len(l), n)]

def bin_data_into_buckets(data, batch_size):
    buckets = []
    size_to_data_dict = {}
    for item in data:
        seq_length = len(item['graph']['vectors'])
        question_length = len(item['question_vectors'])
        try:
            size_to_data_dict[(seq_length, question_length)].append(item)
        except:
            size_to_data_dict[(seq_length, question_length)] = [item]
    for key in size_to_data_dict.keys():
        data = size_to_data_dict[key]
        chunks = get_chunks(data, batch_size)
        for chunk in chunks:
            buckets.append(chunk)
    return buckets
    

def train(data, model, saving_dir, name_prefix, epochs=20, bucket_size=10, trace_every=1):
    

    buckets = bin_data_into_buckets(data, bucket_size)
    start_time = time.time()

    losses = []
    for i in range(epochs):
        random_buckets = sorted(buckets, key=lambda x: random.random())
        sys.stderr.write('--------- Epoch ' + str(i) + ' ---------\n')
        sys.stderr.write('time_elapsed (s): ' + str(time.time() - start_time) + '\n')
        
        for bucket in random_buckets:
            graph_bucket = []
            
#             try:
            for item in bucket:
                node_vectors = item['graph']['vectors']
                types = item['graph']['types']
                A_bw = item['graph']['A_bw']
                y = item['answer']
                item_vector = item['item_vector']
                question_vectors = item['question_vectors']
                question_mask = item['question_mask']
                graph_bucket.append((A_bw, node_vectors, types, item_vector, question_vectors, question_mask, y))
            if len(graph_bucket) > 0:
                loss = model.train(graph_bucket, 1)
                losses.append(loss)
#             except Exception as e:
#                 print('Exception caught during training: ' + str(e))
        if i % trace_every == 0:
            save_filename = saving_dir + name_prefix + '-' + str(i) + '.tf'
            sys.stderr.write('Saving into ' + save_filename + '\n')
            model.save(save_filename)
    return losses
    

In [21]:
n_epochs = 60

print('Loading model: ')
_saving_dir = 'data\\'

losses = train(training_data,
      nn_model_gcn_with_attention,
      _saving_dir,
      name_prefix='qa',
      epochs=n_epochs,
      bucket_size=10,
      trace_every=1,
      )

Loading model: 


--------- Epoch 0 ---------
time_elapsed (s): 8.988380432128906e-05


ValueError: setting an array element with a sequence.

c) 

### !
display loss for 

10 epochs,  25 epochs,   60 epochs

### 4

In [ ]:
def test(data, model):
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0
    for item in data:
        node_vectors = item['graph']['vectors']
        types = item['graph']['types']
        A_bw = item['graph']['A_bw']
        expected = item['answer']
        item_vector = item['item_vector']
        question_vectors = item['question_vectors']
        question_mask = item['question_mask']
#        try:
        prediction = model.predict(A_bw, node_vectors, types, item_vector, question_vectors, question_mask)
        if prediction == expected and expected == _is_relevant:
            true_positives += 1
        if prediction == expected and expected == _is_not_relevant:
            true_negatives += 1
        if prediction != expected and expected == _is_relevant:
            false_negatives += 1
        if prediction != expected and expected == _is_not_relevant:
            false_positives += 1
#        except Exception as e:
#            print('Exception caught during training: ' + str(e))
    try:
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / (true_positives + false_negatives)
        f1 = 2 * 1 / (1 / precision + 1 / recall)
        print('precision', precision)
        print('recall', recall)
        print('f1', f1)
    except:
        print('Cannot compute precision and recall.')


Load the test set, made with the other notebook

The test set should have the same max number of nodes

In [ ]:
import pickle

path_to_test_set = ''
# Store data (serialize)
with open(path_to_test_set, 'rb') as handle:
    test_data = pickle.load(handle)


In [ ]:
test(test_data_,   nn_model_gcn_with_attention)